# Interface
- components implement 'Runnable' protocol
- Common methods include
  - invoke [ainvoke]
  - stream [astream]
  - batch [abatch]
- Common properties:
  - input_schema, output_schema
- common i/o


| component   | input type | output type |
| ------ | ---------------- |----------|
| prompt   | dictionary    | prompt value |
|retriever| single string|list of document |
| LLM    | String, list of message or Prompt value | string|
| chatmodel| string, list of messages or prompt values      | chatmessage |
|tool |string\dictionary|tool dependent|
|output parser|output of llm or chatmodel|parser dependent|



# LCEL

- Runnable support
  - async, batch and streaming support
  - fallbacks
  - parallelism
    - llm calls can be time consumeing
    - any components that can be run in parallel are
  - logging is built in

In [5]:
!pip install pydantic==1.10.8
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
import openai
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
openai.api_key = key

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


In [18]:
prompt = ChatPromptTemplate.from_template(
    "give me very short essay on {topic}"
)

model = ChatOpenAI(openai_api_key = key)

output_parser = StrOutputParser()

In [19]:
chain = prompt | model | output_parser

In [20]:
chain.invoke({"topic": "car"})

'A car is a convenient mode of transportation that allows individuals to travel easily from one place to another. It provides independence, flexibility, and comfort to its users. Cars come in various shapes, sizes, and models, offering a wide range of options to suit different needs and preferences. With advancements in technology, cars have become more efficient, safer, and environmentally friendly. Despite some drawbacks such as traffic congestion and pollution, cars remain a popular choice for many people around the world due to their convenience and practicality.'

In [22]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [29]:
!pip install langchain[docarray]
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00


In [48]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Hency is good person", "cat do drink milk"],
    embedding = OpenAIEmbeddings(openai_api_key = key)
)

retriever = vectorstore.as_retriever()

In [49]:
retriever.get_relevant_documents('who is good boy?')

[Document(page_content='Hency is good person'),
 Document(page_content='cat do drink milk')]

In [50]:
retriever.get_relevant_documents('who drinks milk?')

[Document(page_content='cat do drink milk'),
 Document(page_content='Hency is good person')]

In [51]:
template = 'Answer the question based only on the following context: {context}. Question: {question}'

prompt = ChatPromptTemplate.from_template(template)

In [52]:
from langchain.schema.runnable import RunnableMap

In [64]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question']
}) | prompt| model | output_parser

In [54]:
(output_parser)

StrOutputParser()

In [65]:
chain.invoke({'question': 'who is good person?'})

'Hency'

In [84]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [85]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(openai_api_key = key, temperature = 0.5).bind(functions = functions)

In [86]:
ChatOpenAI?

In [87]:
runnableq = prompt | model

In [95]:
runnableq.invoke({"input": 'dk airport'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"DK"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 60, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-164cf095-cbcd-4e09-9312-cb9e1ad282f5-0')

In [89]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [90]:
model = model.bind(functions = functions)

In [91]:
runnable = prompt | model

In [92]:
runnable.invoke({"input": "airport"})

AIMessage(content='Sure, please provide me with the airport code you would like to get weather information for.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 92, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89e06e7f-e8d3-4f46-a26f-23e7e0271eff-0')

In [94]:
runnable.invoke({"input": "football"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"football"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 92, 'total_tokens': 108}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e47c03fe-6652-45d1-9778-640690675376-0')

# Fallbacks


In [100]:
from langchain.llms import OpenAI
import json

In [101]:
simple_model = OpenAI(openai_api_key = key,
                      temperature = 0.5,
                      max_tokens = 100,
                      )

simple_chain = simple_model | json.loads

In [102]:
challenge = 'write three poems in a json blob, where each poem is a json'

In [105]:
simple_model.invoke(challenge)

' object\n\n\n1) "Autumn Leaves"\n\n{\n    "title": "Autumn Leaves",\n    "author": "Jane Smith",\n    "content": "The leaves are changing colors,\n                From green to red and gold,\n                A symphony of autumn,\n                As the breeze starts to unfold.\n                Each leaf a work of art,\n                A masterpiece in its own right,\n                Falling gently to the ground,\n                In a graceful, effortless flight.\n                And as I walk through the'

In [106]:
# here simple_chain.invoke(challenge) would provide error

In [117]:
model = ChatOpenAI(openai_api_key = key, temperature = 0.5)
chain = model | StrOutputParser()

a = chain.invoke(challenge)
a

'{\n  "poem1": {\n    "title": "The Whispering Wind",\n    "author": "Unknown",\n    "lines": [\n      "The whispering wind in the trees",\n      "Carries secrets on its breeze",\n      "A soft melody that never ceases",\n      "Bringing peace to all who listens"\n    ]\n  }\n}\n\n{\n  "poem2": {\n    "title": "Dancing in the Rain",\n    "author": "Emily Dickinson",\n    "lines": [\n      "Dancing in the rain, so free",\n      "Letting go of all that binds me",\n      "Feeling the drops on my skin",\n      "Washing away all my sin"\n    ]\n  }\n}\n\n{\n  "poem3": {\n    "title": "The Night Sky",\n    "author": "Robert Frost",\n    "lines": [\n      "The night sky, a canvas so vast",\n      "With stars that twinkle, never to last",\n      "A reminder of our place in the universe",\n      "A beauty that never fails to immerse"\n    ]\n  }\n}'

In [118]:
sa = json.loads(a)
sa

JSONDecodeError: Extra data: line 14 column 1 (char 268)

In [113]:
model = ChatOpenAI(openai_api_key = key, temperature = 0.5)
chain = model | StrOutputParser() | json.loads


In [114]:
chain.invoke(challenge)

JSONDecodeError: Extra data: line 14 column 1 (char 276)

In [ ]:
#fallback ko kuro chai alli bujhina hai